In [1]:
!wget -O data.zip "https://kth-project.s3.eu-north-1.amazonaws.com/data.zip"

--2019-12-24 22:39:19--  https://kth-project.s3.eu-north-1.amazonaws.com/data.zip
Resolving kth-project.s3.eu-north-1.amazonaws.com (kth-project.s3.eu-north-1.amazonaws.com)... 52.95.169.20
Connecting to kth-project.s3.eu-north-1.amazonaws.com (kth-project.s3.eu-north-1.amazonaws.com)|52.95.169.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 345929473 (330M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 329.90M  18.4MB/s    in 19s     

2019-12-24 22:39:39 (17.2 MB/s) - ‘data.zip’ saved [345929473/345929473]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/DNAString.txt      
  inflating: data/DNAString_validation.txt  
  inflating: data/inserts.txt        
  inflating: data/inserts_validation.txt  
  inflating: data/labels.txt         
  inflating: data/labels_validation.txt  
  inflating: data/positions.txt      


In [0]:
import pandas as pd
df=pd.read_csv('data/inserts.txt', sep=',',header=None) 

In [0]:
labelsdf=pd.read_csv('data/labels.txt', sep=',',header=None)

In [5]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution() # define eager execution to be able to preview our training data

In [0]:
# Split traning data into equal parts
data_chunks = list(np.split(df.values, 272)) #5867
labels_chunks = list(np.split(labelsdf.values, 272))

In [0]:
# define a generator to reshape our dataset and zip it to be more manageable
batchSize=data_chunks[0].shape[0]
def genenerator():
    for i, j in zip(data_chunks, labels_chunks):
        yield i.reshape([batchSize,4,200,-1]), j
#create a tensor for our training dataset
train_dataset = tf.data.Dataset.from_generator(genenerator,output_shapes=([batchSize,4,200,1],[batchSize,1]),output_types=(tf.float32, tf.float32)) 



In [0]:
#import keras API and matplotlib
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [0]:
#create our neural network model
tf.keras.backend.clear_session()
regularization=tf.keras.regularizers.l2(0.01)

model = models.Sequential()
model.add(layers.Conv2D(100, (4, 30), activation='relu', input_shape=(4, 200,1), kernel_regularizer=regularization))
model.add(layers.MaxPooling2D((3, 10),strides=1,padding='same'))
model.add(layers.Conv2D(50, (1, 10), padding='same', activation='relu',kernel_regularizer=regularization))
model.add(layers.MaxPooling2D((1, 1),strides=1))
model.add(layers.Conv2D(50, (1, 5), activation='relu',kernel_regularizer=regularization))
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu',kernel_regularizer=regularization))
model.add(layers.Dense(1, activation=tf.nn.sigmoid))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 171, 100)       12100     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 171, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 171, 50)        50050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 171, 50)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 167, 50)        12550     
_________________________________________________________________
flatten (Flatten)            (None, 8350)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                4

In [0]:
#compile our model
model.compile(optimizer= tf.train.AdamOptimizer(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
#train our neural network
#use batch training
#https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/
for trainData, trainLabels in iter(train_dataset):
 model.fit(trainData, trainLabels,batch_size=batchSize, epochs=5)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 5867 samples
Epoch 1/5
5867/5867 [==============================] - 9s 1ms/sample - loss: 2.8783 - acc: 0.4994
Epoch 2/5
5867/5867 [==============================] - 0s 29us/sample - loss: 2.7329 - acc: 0.4999
Epoch 3/5
5867/5867 [==============================] - 0s 29us/sample - loss: 2.5911 - acc: 0.5287
Epoch 4/5
5867/5867 [==============================] - 0s 29us/sample - loss: 2.4617 - acc: 0.5003
Epoch 5/5
5867/5867 [==============================] - 0s 29us/sample - loss: 2.3392 - acc: 0.5011
Train on 5867 samples
Epoch 1/5
5867/5867 [==============================] - 0s 29us/sample - loss: 2.2244 - acc: 0.5250
Epoch 2/5
5867/5867 [==============================] - 0s 29us/sample - loss: 2.1169 - acc: 0.5570
Epoch 3/5
5867/5867 [==============================] - 0s 29us/sample - loss: 2.0166 - acc: 0.5647
Epoch 4/5
5867/5867 [==============================] - 0s 29us/sample -

In [0]:
#import validation data
labelsValidationDf=pd.read_csv('data/labels_validation.txt', sep=',',header=None)
dataValidationDf=pd.read_csv('data/inserts_validation.txt', sep=',',header=None)

In [0]:
# define a generator to reshape our dataset and zip it to be more manageable
validationSetSize=len(labelsValidationDf)
def geneneratorValidation():
        npArr = np.array(dataValidationDf.values)
        yield npArr.reshape([validationSetSize,4,200,-1]), labelsValidationDf
#create a tensor for our training dataset
validation_dataset = tf.data.Dataset.from_generator(geneneratorValidation,output_shapes=([validationSetSize,4,200,1],[validationSetSize,1]),output_types=(tf.float32, tf.float32)) 



In [0]:
validationData, validationLabels = next(iter(validation_dataset))

In [19]:
#Validate our model against validation data
validation_loss, validation_accuracy = model.evaluate(validationData, validationLabels)
print('Validation accuracy:', validation_accuracy)

177482/177482 [==============================] - 10s 56us/sample - loss: 0.6266 - acc: 0.6880
Validation accuracy: 0.6879909
